<a href="https://colab.research.google.com/github/sergioGarcia91/GAIRA_UIS_2025/blob/main/03_GAIRA_Predecir_YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción

En este Notebook se utilizarán diferentes modelos reentrenados de [YOLOv8](https://docs.ultralytics.com/models/yolov8/) para **clasificar imágenes de rocas** según su tipo: **ígneas**, **sedimentarias** o **metamórficas**.
El objetivo es evaluar y comparar el desempeño de estos modelos, considerando las diferencias en sus datos de entrenamiento y configuraciones, y así analizar cómo varían sus predicciones en tareas de segmentación geológica.

> **Observación:**  
>  
> Para el proceso de predicción, el uso de recursos computacionales no es tan exigente como en la etapa de entrenamiento de un modelo YOLOv8. Dado que las predicciones se realizarán sobre un directorio que contiene imágenes estáticas de interés, **es posible utilizar la `CPU` sin inconvenientes**.  
>  
> No obstante, si se emplea un valor alto de `imgsz` (tamaño de imagen), la **CPU estándar podría no ser suficiente**. En ese caso, se recomienda utilizar la **`TPU v2-8`**, la cual ofrece mayor capacidad de **RAM**, permitiendo el procesamiento de imágenes redimensionadas a una resolución más alta.  
>  
> Por otro lado, si se desea realizar inferencia sobre **video en tiempo real o streaming**, se recomienda el uso de una **`GPU`**. Puede iniciarse con la **`GPU T4`**, que es una de las opciones más ligeras en cuanto a consumo de recursos. Si el rendimiento no es suficiente, se puede escalar a una GPU más potente según sea necesario.




# Inicio

In [ ]:
# Instalar ultralytics
!pip install ultralytics

## Conexión con Google Drive

In [ ]:
# Para acceder a archivos almacenados en tu unidad de Google Drive desde este Notebook
from google.colab import drive

# Monta tu Google Drive en el directorio /content/drive
drive.mount('/content/drive')

In [ ]:
## Carga de librerías
import os
import shutil
import random
import ultralytics

In [ ]:
# Verifica la instalación y disponibilidad de GPU
ultralytics.checks()

## Funciones

En la siguiente celda de código se definen una serie de funciones que permiten simplificar y automatizar el proceso de predicción utilizando los modelos YOLOv8 previamente reentrenados. Estas funciones incluyen:

- Descarga y extracción de modelos desde Google Drive.
- Descarga y extracción del conjunto de imágenes de prueba.
- Ejecución de predicciones por lote utilizando múltiples modelos.

Este conjunto de utilidades facilita la comparación del desempeño entre diferentes modelos y configuraciones de entrenamiento.


In [ ]:
#@title utils
import gdown
import zipfile
import os
import shutil

# Descarga y extracción de modelos desde Google Drive
def get_models_from_drive(
    file_id="1O0GzwHicc4T2GOrVyZp5gytLpXPr6keN",
    output_zip="archivo.zip",
    extract_to=".",
    modelos_dir="Modelos_YOLOv8_Gaira"
    ):
    print('Descargando archivo desde Google Drive...')
    # https://drive.google.com/file/d/1O0GzwHicc4T2GOrVyZp5gytLpXPr6keN/view?usp=sharing
    try:
        gdown.download(f"https://drive.google.com/uc?id={file_id}", output=output_zip, quiet=False)
    except Exception as e:
        print(f"Error al descargar: {e}")
        return None, None

    print('Extrayendo archivo ZIP...')
    if os.path.exists(output_zip):
        try:
            with zipfile.ZipFile(output_zip, 'r') as zip_ref:
                zip_ref.extractall(extract_to)
            print('Archivos extraídos correctamente.')
        except zipfile.BadZipFile:
            print("Error: el archivo ZIP está corrupto.")
            return None, None
    else:
        print(f"Archivo ZIP no encontrado: {output_zip}")
        return None, None

    print('Eliminando archivo ZIP...')
    os.remove(output_zip)

    print('Buscando archivos .pt en el directorio de modelos...')
    modelos_path = os.path.join(extract_to, modelos_dir)
    if not os.path.exists(modelos_path):
        print(f"Directorio no encontrado: {modelos_path}")
        return None, None

    lista_modelos = sorted([
        archivo for archivo in os.listdir(modelos_path)
        if archivo.endswith('.pt')
    ])

    print(f'Se encontraron {len(lista_modelos)} modelos.')
    return modelos_path, lista_modelos

#Descarga y extracción del conjunto de imágenes de prueba
def get_image_test_from_drive(
    file_id="1QxVlnk0fV3pxl6cxXYdpPVDZMyrF7eY1",
    output_zip="archivo.zip",
    extract_to=".",
    images_dir="Muestras_test"
    ):
    print('Descargando conjunto de imágenes de prueba...')
    # https://drive.google.com/file/d/1QxVlnk0fV3pxl6cxXYdpPVDZMyrF7eY1/view?usp=sharing
    try:
        gdown.download(f"https://drive.google.com/uc?id={file_id}", output=output_zip, quiet=False)
    except Exception as e:
        print(f"Error al descargar el archivo: {e}")
        return None, None

    print('Extrayendo archivo ZIP...')
    if os.path.exists(output_zip):
        try:
            with zipfile.ZipFile(output_zip, 'r') as zip_ref:
                zip_ref.extractall(extract_to)
            print('Archivos extraídos correctamente.')
        except zipfile.BadZipFile:
            print("Error: el archivo ZIP está corrupto.")
            return None, None
    else:
        print(f"No se encontró el archivo ZIP: {output_zip}")
        return None, None

    print('Eliminando archivo ZIP...')
    os.remove(output_zip)

    print('Obteniendo rutas de imágenes...')
    images_path = os.path.join(extract_to, images_dir)
    if not os.path.exists(images_path):
        print(f"El directorio no existe: {images_path}")
        return None, None

    lista_images = sorted(os.listdir(images_path))
    print(f'Se encontraron {len(lista_images)} imágenes.')

    return images_path, lista_images

# Ejecución de predicciones por lote utilizando múltiples modelos
def predecir_images(path_modelos,
                    lista_modelos,
                    path_image,
                    path_save,
                    imgsz_as_dict=False,
                    imgsz_dict=dict(),
                    imgsz=[1536, 1280],
                    conf=0.01,
                    verbose=True,
                    save=True,
                    save_txt=True,
                    save_conf=True,
                    iou=0.7,
                    max_det=300,
                    **kwargs):  # Otros argumentos de ultralytics.YOLO.predict
    """
    Realiza predicciones sobre un conjunto de imágenes utilizando varios modelos YOLOv8.

    Parámetros:
    - path_modelos: ruta donde están almacenados los modelos .pt
    - lista_modelos: lista de nombres de archivos .pt
    - path_image: ruta del directorio con imágenes a predecir
    - path_save: ruta donde se guardarán los resultados
    - imgsz_as_dict: si True, usa imgsz_dict por modelo
    - imgsz_dict: diccionario {modelo: tamaño_imagen}
    - imgsz: tamaño por defecto si imgsz_as_dict es False
    - conf: umbral de confianza
    - verbose, save, save_txt, save_conf, iou, max_det: parámetros para el predictor YOLOv8
    """

    for modelo in lista_modelos:
        ruta_modelo = os.path.join(path_modelos, modelo)

        # Determinar tamaño de imagen
        tamaño = imgsz_dict.get(modelo, imgsz) if imgsz_as_dict else imgsz

        try:
            print('## ---------- ##')
            print(f"Cargando modelo: {modelo}")
            model = ultralytics.YOLO(ruta_modelo)

            print("Ejecutando predicción...\n")
            results = model.predict(
                source=path_image,
                imgsz=tamaño,
                conf=conf,
                project=path_save,
                name=modelo.replace('.pt', ''),
                verbose=verbose,
                save=save,
                save_txt=save_txt,
                save_conf=save_conf,
                iou=iou,
                max_det=max_det,
                **kwargs
            )

        except Exception as e:
            print(f"Error con el modelo '{modelo}': {e}")
            print("Saltando al siguiente modelo...\n")
            continue


# Descargar modelos

In [ ]:
ruta_modelos, lista_modelos = get_models_from_drive()

In [ ]:
ruta_modelos

**Nota sobre los modelos utilizados**

Algunos de los modelos YOLOv8 utilizados en este Notebook corresponden a la investigación desarrollada por:

**García-Arias, S. y Velandia, F. A. (2025).**  
*Modelos de redes neuronales convolucionales como herramienta para automatizar la clasificación de rocas.*  
**Revista EIA**, 22(43), Art. 4317, pp. 1–28.  
https://doi.org/10.24050/reia.v22i43.1813


In [ ]:
lista_modelos

# Imagenes Test

In [ ]:
ruta_imagenes, lista_imagenes = get_image_test_from_drive()

In [ ]:
ruta_imagenes

In [ ]:
lista_imagenes

# Predicciones

Los modelos YOLOv8 cuentan con varios atributos que pueden ser modificados durante el proceso de predicción.  
A continuación, se hará uso únicamente de algunos de ellos, adecuados a nuestro caso particular de segmentación de rocas. Para más información y una descripción completa de los argumentos disponibles, puedes consultar la documentación oficial de Ultralytics sobre [Predict](https://docs.ultralytics.com/modes/predict/#inference-arguments).


In [ ]:
# Ruta donde se guardarán los resultados de las predicciones realizadas por los modelos
ruta_guardar_predicciones = '/content/drive/MyDrive/Manuscritos_Investigacion/Rocas/Predicciones_GAIRA'

In [ ]:
# Diccionario para especificar el tamaño de imagen (imgsz) que se debe usar durante la predicción
# según el modelo YOLOv8 utilizado. Valores más altos requieren más memoria (GPU o TPU recomendada)
diccionario_size_img = {'rocas_GAIRA_01-n.pt':[1536, 1280],
                        'rocas_GAIRA_02-n.pt':[1536, 1280],
                        #'rocas_GAIRA_03-n.pt':[3072, 2560], # Recomendable usar TPU v2-8
                        'rocas_GAIRA_04-n.pt':[1536, 1280],
                        'rocas_GAIRA_05-n.pt':640,
                        'rocas_GAIRA_06-n.pt':640}

In [ ]:
predecir_images(path_modelos= ruta_modelos,
                lista_modelos= lista_modelos,
                path_image= ruta_imagenes,
                path_save= ruta_guardar_predicciones,
                imgsz_as_dict = True,
                imgsz_dict= diccionario_size_img,
                imgsz= 640,
                conf= 0.010,
                verbose= True,
                save= True,
                save_txt= True,
                save_conf= True,
                iou= 0.7,
                max_det= 300,
                )

# Fin